# Notebook CO2 emisiones


Explorare una planilla de datos que contiene segun su año y segun pais la emision de CO2 que estan representadas por cada persona en el pais.

---



## Importo mis librerias a usar


In [ ]:
import pandas as pd

# Carga de datos

In [ ]:
# en este caso cargare desde drive los datos
from google.colab import drive
drive.mount('/content/drive')

datos = '/content/drive/MyDrive/data-analist/per-capita-co-emissions.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# cargos mis datos en un Dataframe
df_CO2 = pd.read_csv(datos)
# mustro los datos cargados

df_CO2.head()


,Entity,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,1949,0.001992
1,Afghanistan,1950,0.010837
2,Afghanistan,1951,0.011625
3,Afghanistan,1952,0.011468
4,Afghanistan,1953,0.013123


## Análisis del promedio entre todos sus años



In [ ]:
# busco los valores importantes y responderme preguntas como:
# ¿cuanto es el promedio de la emision anual per capita de todos los años segun pais?
df_CO2_annual2entity = pd.DataFrame(df_CO2.copy())
df_CO2_annual2entity = df_CO2_annual2entity.groupby(['Entity'])['Annual CO₂ emissions (per capita)'].mean().reset_index()

df_CO2_annual2entity.head()


,Entity,Annual CO₂ emissions (per capita)
0,Afghanistan,0.145672
1,Africa,0.562829
2,Albania,1.282190
3,Algeria,1.663785
4,Andorra,6.709084


In [ ]:
# representemos en un grafico
import plotly.express as px
fig = px.scatter(df_CO2_annual2entity, x='Entity', y='Annual CO₂ emissions (per capita)', color='Entity')
fig.show()

como se puede observar en el anterior grafico se puede observar cual ha sido el valor promedio de emision de CO2 por cada pais en todos sus años. Pero este grafico muestra los  valores por cada pais lo cual no es visualmente muy entendible. verificare cuantos valores iguales de CO2 comparten entre paises con pandas.


In [ ]:
df_CO2_annual2entity['Annual CO₂ emissions (per capita)'].value_counts()

,count
Annual CO₂ emissions (per capita),
0.145672,1
0.562829,1
1.282190,1
1.663785,1
6.709084,1
...,...
1.858346,1
3.276408,1
0.573742,1


Se puede observar que los valores son distintos entre cada pais lo cual se debera por las milesimas que se toman en cuenta. Una solucion seria bajar a dos o una unidad despues de la coma en todos los valores de CO2.

In [ ]:
df_CO2_annual2entity_round = pd.DataFrame(df_CO2_annual2entity.copy())
df_CO2_annual2entity_round['Annual CO₂ emissions (per capita)'] = df_CO2_annual2entity['Annual CO₂ emissions (per capita)'].round(1)
df_CO2_annual2entity_round.head()

,Entity,Annual CO₂ emissions (per capita)
0,Afghanistan,0.1
1,Africa,0.6
2,Albania,1.3
3,Algeria,1.7
4,Andorra,6.7


In [ ]:
df_CO2_annual2entity_round['Annual CO₂ emissions (per capita)'].value_counts()

,count
Annual CO₂ emissions (per capita),
0.1,19
0.3,13
0.5,11
0.2,10
0.7,9
...,...
4.3,1
7.3,1
28.9,1



> En este caso con 1 milesima obtendremos mas valores iguales de CO2 pero se perdera la precision de los valores. realizaremos un grafico con el mismo.



In [ ]:
df_CO2_annual2entity_round = df_CO2_annual2entity_round.groupby('Annual CO₂ emissions (per capita)')['Entity'].count().reset_index()
df_CO2_annual2entity_round.head()


,Annual CO₂ emissions (per capita),Entity
0,0.0,1
1,0.1,19
2,0.2,10
3,0.3,13
4,0.4,8


In [ ]:
fig = px.density_heatmap(df_CO2_annual2entity_round, y='Annual CO₂ emissions (per capita)',x='Entity', color_continuous_scale="Viridis", text_auto=True)
fig.show()

otro grafico seria el de puntos.

In [ ]:
fig = px.scatter(df_CO2_annual2entity_round, x='Entity', y='Annual CO₂ emissions (per capita)', color='Annual CO₂ emissions (per capita)', color_continuous_scale='VIridis')
fig.show()

Conclusion hay muchos paises con diferentes valores unicos de emisiones de co2 por lo que seria mejor realizar un grafico geografico para el mismo. Para represntar mejor los valores. Los valores mas altos son los de los paises de menos poblacion y que tienen mas emisones de CO2.

In [ ]:
fig = px.choropleth(df_CO2_annual2entity,
                    locations="Entity",
                    locationmode='country names',
                    color="Annual CO₂ emissions (per capita)", # Columna que contiene los valores de emisión de CO2
                    hover_name="Entity", # Muestra el nombre del país al pasar el mouse
                    color_continuous_scale="Spectral", # Escala de color para representar los valores
                    title='Emisiones Promedio Anuales de CO2 per Cápita por País')
fig.show()


## Análisis del pais por cada año de las emisiones

In [ ]:
#comienzo definiendo una funcion que tome una variable que sera introducida por el usuario "pais"
def esPais(paisDefinido):
  df_CO2_pais = pd.DataFrame(df_CO2.copy())
  df_CO2_pais['Entity'].value_counts()
  if paisDefinido in df_CO2_pais['Entity'].values:
    return True
  else:
    return False
def paisSeleccionado(pais):
  df_CO2_pais = pd.DataFrame(df_CO2.copy())
  if esPais(pais) == True:
    df_CO2_pais = df_CO2_pais.loc[df_CO2_pais['Entity'] == pais]
    return df_CO2_pais
  else:
    print('El pais no se encuentra por defecto se seleccionara Argentina')
    df_CO2_pais = df_CO2_pais.loc[df_CO2_pais['Entity'] == "Argentina"]
    return df_CO2_pais

In [ ]:
# llamo a la funcion cuando el usuario quiera recuperar un pais en especifico
while True:
  pais = paisSeleccionado(str(input()))
  if pais is not None:
    break
# represento dicho pais en un grafico individual
if pais is not None:
  fig = px.line(pais, x='Year', y='Annual CO₂ emissions (per capita)', color='Entity')
  fig.show()


ss
El pais no se encuentra por defecto se seleccionara Argentina


## Comparacion entre paises

aqui reutilizaria la funcion antes realizada y la llamaria 2 veces

In [ ]:
# seleccion de los dos paises a comparar
while True:
  pais1 = paisSeleccionado(str(input("Ingrese el pais 1:")))
  pais2 = paisSeleccionado(str(input("Ingrese el pais 2:")))
  if pais1 is not None and pais2 is not None:
    break
df_paises = pd.concat([pais1, pais2])
# realizo los graficos de los dos paises
fig = px.line(df_paises, x='Year', y='Annual CO₂ emissions (per capita)', color='Entity')
fig.show()

Ingrese el pais 1:Cuba
Ingrese el pais 2:
El pais no se encuentra por defecto se seleccionara Argentina
